<a href="https://colab.research.google.com/github/yakovsushenok/Thesis/blob/main/BirdSongIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import zipfile
import os

# extracting the training data (audio files) from the zip file (12 minutes)
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_short_audio.zip', 'r')
zip_ref.extractall('/content/tmp') 
zip_ref.close()

In [ ]:
# extracting the data which has audio files that are of similar length to the testing data
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/train_soundscapes.zip', 'r')
zip_ref.extractall('/content/trainSoundscapes') 
zip_ref.close()
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/train_metadata.csv") # the metadata
df['relative_path'] = '/content/tmp/' + df['primary_label'] + '/' + df['filename'] 
df = df[['relative_path', 'primary_label']]

# Utility classes

In [ ]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import librosa

class AudioUtil():
  """
  This class will be for various functions that I will use for the audio data
  """
  @staticmethod
  def open(audio_file):
    """
    The load method loads the audio file and produces 2 outputs:
    - waveform (Tensor): A waveform is a graphical representation of a sound wave as it moves through a medium over time
    - sample rate (int): In audio production, a sample rate defines how many times per second a sound is sampled. Technically speaking, it is the frequency of samples used in a digital recording.
    The standard sample rate used for audio CDs is 44.1 kilohertz (44,100 hertz). That means each second of a song on a CD contains 44,100 individual samples.
    """
    waveform, samplerate = torchaudio.load(audio_file) # https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html#loading-audio-data-into-tensor
    return (waveform, samplerate) # right now not sure whether the 
  



In [ ]:

def numOfChannels(filename):

  audioData, _ = librosa.load(filename)
  #print(True if len(audioData.shape) == 1 else False )
  return True if len(audioData.shape) == 1 else False 
df['numChan'] = df['relative_path'].apply(numOfChannels)
pd.value_counts(df.numChan)